In [1]:
import pandas as pd 
import numpy as np

from googletrans import Translator
translator = Translator()

def to_float(x):
    if type(x) == str and "," in x:
        x = x.replace(",", ".")
        return float(x)
    return x

In [2]:
def clean_data(clean_guide):
    filename = clean_guide["filename"]
    genesis_table = clean_guide["Table"],
    column_names = clean_guide["Columns"]
    column_names = column_names.split(", ")
    skiprows = clean_guide['Read Lines'][0] - 1
    nrows = clean_guide['Read Lines'][1] - skiprows
    na_values = clean_guide['NA']
    to_translate = clean_guide["To Translate"]

    df = pd.read_csv(filename, sep=";", encoding="latin-1", header=None, names=column_names, skiprows=skiprows, nrows=nrows, na_values=na_values)

    # Convert to numeric
    for col in column_names[clean_guide['to_numeric']:]:
        df[col] = df[col].apply(lambda x: to_float(x))

    if to_translate:
        for k, v in to_translate.items():
            if v: # can be optimized
                unique_vals = list(df[k].unique())
                translated_unique_vals = [translator.translate(x).text for x in unique_vals]
                mappings = dict(zip(unique_vals, translated_unique_vals))
                df[k] = df[k].apply(lambda x: mappings[x])
                df[k] = df[k].str.title()
            else:
                df[k] = df[k].apply(lambda x: translator.translate(x).text)
    
    return df

In [ ]:
d1 = {
    "filename": "Sales Packaging-companies-end consumers-federal states-years.csv",
    "Table": "32136-0010",
    "Columns": "Year, State, Company, Sales packaging. v. private end users (1000t), Sales packaging. v. private end use per inhabitant (kg)",
    "Read Lines": [7, 182],
    "To Translate": {'State':1},
    "NA": [".", "-", "x"],
    "to_numeric": 2
}
d2 = {
    "filename": "Sales Packaging-companies-end consumers-years-return types.csv",
    "Table": "32136-0002",
    "Columns": "Year, Return Method, Unit, Sales Packaging After Sorting (Industrial Solutions), Sales Packaging After Sorting (System Operators), Overall Sales Packaging After Sorting",
    "Read Lines": [9, 63],
    "NA": [".", "x", "-"],
    "To Translate": {'Return Method': 0, 'Unit': 1},
    "to_numeric": 3
}
d3 = {
    "filename": "Sales Packaging-companies-end consumers-years.csv",
    "Table": "32136-0001",
    "Columns": "KPI, Unit, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020",
    "Read Lines": [8,12],
    "NA": [".", "-", "x"],
    "To Translate": {"KPI": 0},
    "to_numeric": 2
}
d4 = {
    "filename": "Sales Packaging-end consumers-federal states-years-type of sales.csv",
    "Table": "32136-0011",
    "Columns": "Year, State, Mixed packaging, Packaging made of paper & cardboard & carton, Mixed glass (colored & mixed glass), Glass collected separately in color, Plastics collected separately, Metals collected separately, Composites collected separately, Total",
    "Read Lines": [8, 183],
    "NA": [".", "-"],
    "To Translate": {"State":1},
    "to_numeric": 2
}